In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import varie e inizializzazione delle variabili dei modelli e del tuner

In [62]:
from sklearn.model_selection import train_test_split
from IRESNs_tensorflow.time_series_datasets import *
from IRESNs_tensorflow.models import ESN, IRESN, IIRESN, IIRESNvsr
from tensorflow import keras

PROJECT_ROOT = os.path.abspath(os.getcwd())
DATASET_NAME = "CharacterTrajectories"
DATASET_DIR = os.path.join(PROJECT_ROOT, "datasets")

READOUT_ACTIVATION_BINARY = keras.activations.sigmoid
LOSS_FUNCTION_BINARY = keras.losses.BinaryCrossentropy()

READOUT_ACTIVATION = keras.activations.softmax
LOSS_FUNCTION = keras.losses.SparseCategoricalCrossentropy()

# Dataset dependent settings
RESERVOIRS = 3
OUTPUT_UNITS = 20

# Tuner settings
MAX_EPOCHS = 3
PATIENCE = 1
MAX_TRIALS = 5
OVERWRITE = False
TRIALS = 1

BENCHMARKS_TRIALS = 1

MINVAL = 0.01
MAXVAL = 1.5

Definizione della funzione utilizzata dal tuner per istanziare i modelli per la model selection.
Si definisce un modello ESN.

In [63]:
def build_ESN(tuner):
    ESN_model = ESN(units=100,
                    connectivity=tuner.Float('connectivity', min_value=0.01, max_value=1.),
                    spectral_radius=tuner.Float('spectral radius', min_value=MINVAL, max_value=MAXVAL),
                    input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                    bias_scaling=tuner.Float('bias scaling', min_value=MINVAL, max_value=MAXVAL),
                    leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                    output_units=OUTPUT_UNITS,
                    readout_activation=READOUT_ACTIVATION,
                    )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    ESN_model.compile(
        optimizer=keras.optimizers.Adam(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )

    return ESN_model

In [64]:
def build_IRESN(tuner):
    connectivity = [tuner.Float('connectivity ' + str(i), min_value=0.01, max_value=1.) for i in range(RESERVOIRS)]
    sr = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in range(RESERVOIRS)]

    IRESN_model = IRESN(units=100,
                        sub_reservoirs=RESERVOIRS,
                        connectivity=connectivity,
                        spectral_radius=sr,
                        input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                        bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                        leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                        output_units=OUTPUT_UNITS,
                        readout_activation=READOUT_ACTIVATION,
                        )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IRESN_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IRESN_model

In [65]:
def build_IIRESN(tuner):
    sr = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in range(RESERVOIRS)]

    """
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     tuner.Float('connectivity ' + str(i) + '->' + str(j), min_value=0., max_value=1.)
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    off_diagonal_limits = [[0. if i == j else
                     tuner.Float('limit ' + str(i) + '->' + str(j), min_value=0., max_value=1.)
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]
    """
    limit = tuner.Float('limit X->Y', min_value=0., max_value=1.)
    off_diagonal_limits = [[0. if i == j else limit  # 0 sulla diagonale limit su tutte le posizioni off-diagonali
                            for i in range(RESERVOIRS)]
                           for j in range(RESERVOIRS)]

    fixed = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     fixed
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    IIRENS_model = IIRESN(units=100,
                          sub_reservoirs=RESERVOIRS,
                          connectivity=connectivity,
                          spectral_radius=sr,
                          off_diagonal_limits=off_diagonal_limits,
                          input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                          bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                          leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                          output_units=OUTPUT_UNITS,
                          readout_activation=READOUT_ACTIVATION,
                          )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRENS_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRENS_model

In [66]:
def build_IIRESNvsr(tuner):
    sr = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL) for i in range(RESERVOIRS)]

    """
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     tuner.Float('connectivity ' + str(i) + '->' + str(j), min_value=0., max_value=1.)
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    off_diagonal_limits = [[0. if i == j else
                     tuner.Float('limit ' + str(i) + '->' + str(j), min_value=0., max_value=1.)
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]
    """

    fixed = tuner.Float('connectivity X->Y', min_value=0., max_value=1.)
    connectivity = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) if i == j else
                     fixed
                     for i in range(RESERVOIRS)]
                    for j in range(RESERVOIRS)]

    limit = tuner.Float('limit X->Y', min_value=0., max_value=1.)
    off_diagonal_limits = [[0. if i == j else limit  # 0 sulla diagonale limit su tutte le posizioni off-diagonali
                            for i in range(RESERVOIRS)]
                           for j in range(RESERVOIRS)]

    partitions = [tuner.Float('partition ' + str(i), min_value=0.1, max_value=1.0) for i in range(RESERVOIRS)]
    total = sum(partitions)
    # Normalize the partition vector now sum(partitions) == 1.
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))

    IIRESNvsr_model = IIRESNvsr(units=100,
                                partitions=partitions,
                                sub_reservoirs=RESERVOIRS,
                                connectivity=connectivity,
                                off_diagonal_limits=off_diagonal_limits,
                                spectral_radius=sr,
                                input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                                bias_scaling=tuner.Float('bias_scaling', min_value=MINVAL, max_value=MAXVAL),
                                leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                                output_units=OUTPUT_UNITS,
                                readout_activation=READOUT_ACTIVATION,
                                )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    IIRESNvsr_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return IIRESNvsr_model

In [67]:
train_path = os.path.join(DATASET_DIR, DATASET_NAME, DATASET_NAME + '_TRAIN.ts')
test_path = os.path.join(DATASET_DIR, DATASET_NAME, DATASET_NAME + '_TEST.ts')

x_train_all, y_train_all = load_sktime_dataset(train_path)
x_test, y_test = load_sktime_dataset(test_path)

x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                                  test_size=0.33, random_state=42, stratify=y_train_all)

train_set = (x_train, y_train)
val_set = (x_val, y_val)
test_set = (x_test, y_test)

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


In [68]:
from keras_tuner import RandomSearch

working_dir = os.path.join("models", DATASET_NAME)
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

tuner = RandomSearch(
    build_IIRESNvsr,  # build_ESN build_IRESN, build_IIRESN, build_IIRESNvsr
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    seed=42,
    directory=working_dir,
    project_name='IIRESNvsr',  # change this for every model
    overwrite=OVERWRITE,
    executions_per_trial=TRIALS,
)

tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
             callbacks=[
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE),
             ])

best_model_hp = tuner.get_best_hyperparameters()[0]
tf.random.set_seed(42)

for i in range(BENCHMARKS_TRIALS):
    test_model = tuner.hypermodel.build(best_model_hp)
    history = test_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=MAX_EPOCHS,
                             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE,
                                                                      restore_best_weights=True)])
    test_loss, accuracy = test_model.evaluate(x_test, y_test)

    print("     Train accuracy:", history.history['accuracy'][-1])
    print("Validation accuracy:", history.history['val_accuracy'][-1])
    print("      Test accuracy:", accuracy)

INFO:tensorflow:Reloading Oracle from existing project models/CharacterTrajectories/IIRESNvsr/oracle.json
INFO:tensorflow:Reloading Tuner from models/CharacterTrajectories/IIRESNvsr/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
30/30 [==============================] - 0s 3ms/step - loss: 2.7557 - accuracy: 0.1691 - val_loss: 2.4147 - val_accuracy: 0.3298
Epoch 2/3
30/30 [==============================] - 0s 1ms/step - loss: 2.2296 - accuracy: 0.5074 - val_loss: 2.0212 - val_accuracy: 0.5787
Epoch 3/3
45/45 [==============================] - 0s 406us/step - loss: 1.7622 - accuracy: 0.6469
     Train accuracy: 0.5829831957817078
Validation accuracy: 0.6638298034667969
      Test accuracy: 0.6469359397888184


# Build some custom Models

In [69]:
"""
Experimental model with kernel equals to IRESN and recurrent kernel equals to ESN.
"""

from IRESNs_tensorflow.models import ESNInterface
from IRESNs_tensorflow import layer
from IRESNs_tensorflow import initializers


class CustomESN(ESNInterface):
    def __init__(self,
                 units: int,
                 sub_reservoirs: int,
                 output_units: int,
                 reservoir_activation=tf.keras.activations.tanh,
                 readout_activation=tf.nn.tanh,
                 spectral_radius: float = 0.9,
                 connectivity: float = 1.,
                 input_scaling: float = 1.,
                 bias_scaling=None,
                 leaky=0.1,
                 **kwargs
                 ):
        super().__init__(**kwargs)

        kernel_init = initializers.SplitKernel(sub_reservoirs, input_scaling)

        if connectivity == 1.0:
            recurrent_kernel_init = initializers.FullConnected(spectral_radius)
        else:
            recurrent_kernel_init = initializers.RecurrentKernel(connectivity, spectral_radius)

        self.use_bias = bias_scaling is not None
        if self.use_bias:
            bias_init = tf.keras.initializers.RandomUniform(minval=-bias_scaling, maxval=bias_scaling)
        else:
            bias_init = None

        self.reservoir = keras.Sequential([
            keras.layers.Masking(),
            layer.ESN(units, leaky,
                      activation=reservoir_activation,
                      use_bias=self.use_bias,
                      kernel_initializer=kernel_init,
                      recurrent_initializer=recurrent_kernel_init,
                      bias_initializer=bias_init,
                      ),
        ])
        self.readout = keras.Sequential([
            keras.layers.Dense(output_units, activation=readout_activation, name="readout")
        ])


def build_CustomESN(tuner):
    custom_model = CustomESN(units=100,
                          sub_reservoirs=RESERVOIRS,
                          connectivity=tuner.Float('connectivity', min_value=0.01, max_value=1.),
                          spectral_radius=tuner.Float('spectral radius', min_value=MINVAL, max_value=MAXVAL),
                          input_scaling=tuner.Float('input scaling', min_value=MINVAL, max_value=MAXVAL),
                          bias_scaling=tuner.Float('bias scaling', min_value=MINVAL, max_value=MAXVAL),
                          leaky=tuner.Float('leaky', min_value=0.01, max_value=1.0),
                          output_units=OUTPUT_UNITS,
                          readout_activation=READOUT_ACTIVATION
                          )

    alpha = tuner.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    custom_model.compile(
        optimizer=keras.optimizers.Adam(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return custom_model

In [70]:
tuner = RandomSearch(
    build_CustomESN,  # build_ESN build_IRESN, build_IIRESN, build_IIRESNvsr
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    seed=42,
    directory=working_dir,
    project_name='CustomESN',  # change this for every model
    overwrite=OVERWRITE,
    executions_per_trial=TRIALS,
)

tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
             callbacks=[
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE),
             ])

INFO:tensorflow:Reloading Oracle from existing project models/CharacterTrajectories/CustomESN/oracle.json

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
units             |274               |?                 
connectivity      |0.048166          |?                 
spectral radius   |0.61872           |?                 
input scaling     |0.41509           |?                 
bias scaling      |1.3335            |?                 
leaky             |0.35833           |?                 
learning rate     |0.0015582         |?                 



ValueError: Exception encountered when calling layer "sequential" (type Sequential).

In this `tf.Variable` creation, the initial value's shape ((3, 100)) is not compatible with the explicitly supplied `shape` argument ((100, 100)).

Call arguments received:
  • inputs=tf.Tensor(shape=(952, 180, 3), dtype=float64)
  • training=None
  • mask=None